In [179]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings(action='ignore')


### raw data 와 valid data 개요

In [180]:
raw_data = pd.read_csv("../data/night/night_sight_raw.csv", encoding='cp949')
print(raw_data.shape)
raw_data.head()

(1236, 37)


,국가코드,DB종류,특허/실용 구분,문헌종류 코드,발명의 명칭,요약,대표청구항,청구항 수,출원번호,출원일,...,Original CPC Main,Original CPC All,Original IPC Main,Original IPC All,Original US Class Main[US],Original US Class All[US],Original FI[JP],Original F-term[JP],Original Theme Code [JP],WIPS ON key
0,US,US,P,A1,"Non ionic/electrolyte, liquid/gaseous, mechani...",The variable hydraulic press and distillation ...,1. A cool complete vacuum draws any one partic...,4,13/689789,2012-11-30,...,B23P-0013/00,B23P-0013/00,B23P-013/00,B23P-013/00,029/592,029/592,NaN,NaN,NaN,5.414020e+12
1,US,US,P,A1,METHOD AND SYSTEM FOR RECOGNIZING SPACE OF SHO...,Disclosed herein are a method and a system for...,1. A method of recognizing a space of a road s...,18,13/710196,2012-12-10,...,G01S-0013/867,G01S-0013/867 | G01S-0013/862,G01S-013/86,G01S-013/86,342/052,342/052 | 342/070,NaN,NaN,NaN,5.414020e+12
2,US,US,P,B2,Enhanced road vision on full windshield head-u...,A method to selectively project graphical imag...,1. Method to selectively project graphical ima...,17,12/487926,2009-06-19,...,NaN,NaN,B60Q-001/00,B60Q-001/00 | G08G-001/16,340/461,340/461 | 340/435 | 340/436 | 340/905,NaN,NaN,NaN,4.914020e+12
3,US,US,P,A1,METHODS AND SYSTEMS FOR IMAGING AND CUTTING SE...,Methods and systems for imaging and cutting se...,1. A system for singulating microelectronic de...,21,14/093682,2013-12-02,...,B28D-0005/0064,B28D-0005/0064 | B26F-0003/004 | B23K-0026/407...,B28D-005/00,B28D-005/00 | B23K-026/40 | B23Q-015/22 | B26F...,083/076.8,083/076.8 | 083/368 | 451/002 | 219/121.67,NaN,NaN,NaN,5.414010e+12
4,US,US,P,A1,Labels Having an Information Dye,The present invention is directed to labels ha...,1. A label comprising an information dye.,26,13/910719,2013-06-05,...,G09F-0003/02,G09F-0003/02 | B02C-0023/08 | G09F-0003/04 | B...,G09F-003/02,G09F-003/02 | G01N-021/59 | B07C-005/34 | B02C...,241/020,241/020 | 040/299.01 | 040/310 | 209/576 | 356...,NaN,NaN,NaN,5.414010e+12


In [181]:
df = pd.DataFrame(raw_data, columns=raw_data.columns)

In [182]:
#검색식을 통해 수집한 유효특허 후보 데이터
val_data = pd.read_csv("../data/night/night_sight_val.csv", encoding = "cp949")
print(val_data.shape)
val_data.head()

(3191, 7)


,공개(등록)번호,공개(등록)일자,국가,구분,발명의 명칭,IPC,출원인
0,JP-0240608,NaN,JA,공개,LIGHT BEAM SCANNER,G02B,RICOH CO LTD
1,JP-0271064,NaN,JA,공개,OPTICAL SCANNER,G02B,FUJI XEROX CO LTD
2,JP-0220838,NaN,JA,공개,OPTICAL SCANNER AND IMAGE FORMING APPARATUS US...,G02B,RICOH PRINTING SYSTEMS LTD
3,JP-0206021,NaN,JA,공개,OPTICAL SCANNER AND IMAGE FORMING APPARATUS,G02B,RICOH CO LTD
4,JP-0159534,NaN,JA,공개,OPTICAL SCANNER AND IMAGE FORMING APPARATUS,G02B,SHARP CORP


In [183]:
#미국 특허 확인
val_data.tail()

,공개(등록)번호,공개(등록)일자,국가,구분,발명의 명칭,IPC,출원인
3186,04049337,19770920.0,US,등록,Reflex reflecting system for road surface marking,G02B,NaN
3187,04027159,19770531.0,US,등록,Combined use of visible and near-IR imaging sy...,G01J,The
3188,04027159,19770531.0,US,등록,Combined use of visible and near-IR imaging sy...,G01J,The
3189,03961194,19760601.0,US,등록,Tracking and display apparatus for heat-detect...,H01J,Zeiss-Stiftung;
3190,03961194,19760601.0,US,등록,Tracking and display apparatus for heat-detect...,H01J,Zeiss-Stiftung;


### raw data 필요없는 column 제거
* 미국 특허만 활용
* 일본 특허 정보 x  
ex) 'Original FI[JP]', 'Original F-term[JP]', 'Original Theme Code [JP]', '출원인 식별기호[JP]'

In [184]:
df.columns

Index(['국가코드', 'DB종류', '특허/실용 구분', '문헌종류 코드', '발명의 명칭', '요약', '대표청구항', '청구항 수',
       '출원번호', '출원일', '공개번호/공표/재공표 ', '공개일', '등록번호', '등록일', '출원인', '출원인 국적',
       '출원인 수', '출원인 대표명화 코드', '출원인 대표명화 영문명', '출원인 대표명화 국문명[KR]',
       '출원인 식별기호[JP]', '발명자/고안자', '우선권 번호', '우선권 국가', '우선권 주장일', '국제 공개번호',
       '국제 공개일', 'Original CPC Main', 'Original CPC All', 'Original IPC Main',
       'Original IPC All', 'Original US Class Main[US]',
       'Original US Class All[US]', 'Original FI[JP]', 'Original F-term[JP]',
       'Original Theme Code [JP]', 'WIPS ON key'],
      dtype='object')

In [185]:
df = raw_data.dropna(axis=1, how='all')
df.head()

,국가코드,DB종류,특허/실용 구분,문헌종류 코드,발명의 명칭,요약,대표청구항,청구항 수,출원번호,출원일,...,우선권 주장일,국제 공개번호,국제 공개일,Original CPC Main,Original CPC All,Original IPC Main,Original IPC All,Original US Class Main[US],Original US Class All[US],WIPS ON key
0,US,US,P,A1,"Non ionic/electrolyte, liquid/gaseous, mechani...",The variable hydraulic press and distillation ...,1. A cool complete vacuum draws any one partic...,4,13/689789,2012-11-30,...,NaN,NaN,NaN,B23P-0013/00,B23P-0013/00,B23P-013/00,B23P-013/00,029/592,029/592,5.414020e+12
1,US,US,P,A1,METHOD AND SYSTEM FOR RECOGNIZING SPACE OF SHO...,Disclosed herein are a method and a system for...,1. A method of recognizing a space of a road s...,18,13/710196,2012-12-10,...,2012-10-19,NaN,NaN,G01S-0013/867,G01S-0013/867 | G01S-0013/862,G01S-013/86,G01S-013/86,342/052,342/052 | 342/070,5.414020e+12
2,US,US,P,B2,Enhanced road vision on full windshield head-u...,A method to selectively project graphical imag...,1. Method to selectively project graphical ima...,17,12/487926,2009-06-19,...,NaN,NaN,NaN,NaN,NaN,B60Q-001/00,B60Q-001/00 | G08G-001/16,340/461,340/461 | 340/435 | 340/436 | 340/905,4.914020e+12
3,US,US,P,A1,METHODS AND SYSTEMS FOR IMAGING AND CUTTING SE...,Methods and systems for imaging and cutting se...,1. A system for singulating microelectronic de...,21,14/093682,2013-12-02,...,NaN,NaN,NaN,B28D-0005/0064,B28D-0005/0064 | B26F-0003/004 | B23K-0026/407...,B28D-005/00,B28D-005/00 | B23K-026/40 | B23Q-015/22 | B26F...,083/076.8,083/076.8 | 083/368 | 451/002 | 219/121.67,5.414010e+12
4,US,US,P,A1,Labels Having an Information Dye,The present invention is directed to labels ha...,1. A label comprising an information dye.,26,13/910719,2013-06-05,...,NaN,NaN,NaN,G09F-0003/02,G09F-0003/02 | B02C-0023/08 | G09F-0003/04 | B...,G09F-003/02,G09F-003/02 | G01N-021/59 | B07C-005/34 | B02C...,241/020,241/020 | 040/299.01 | 040/310 | 209/576 | 356...,5.414010e+12


In [186]:
df.columns

Index(['국가코드', 'DB종류', '특허/실용 구분', '문헌종류 코드', '발명의 명칭', '요약', '대표청구항', '청구항 수',
       '출원번호', '출원일', '공개번호/공표/재공표 ', '공개일', '등록번호', '등록일', '출원인', '출원인 국적',
       '출원인 수', '출원인 대표명화 코드', '출원인 대표명화 영문명', '발명자/고안자', '우선권 번호', '우선권 국가',
       '우선권 주장일', '국제 공개번호', '국제 공개일', 'Original CPC Main', 'Original CPC All',
       'Original IPC Main', 'Original IPC All', 'Original US Class Main[US]',
       'Original US Class All[US]', 'WIPS ON key'],
      dtype='object')

### valid data에서 US data 외 데이터 drop

In [187]:
val_data = val_data[val_data.국가 == "US"]
val_data = val_data.drop_duplicates(val_data.columns)
val_data.shape
# val_data.to_csv("1.csv", encoding='cp949')

(335, 7)

### raw data와 valid data의 공개/등록 번호 form 맞춰주기

In [188]:
print("<valid> \n", val_data['공개(등록)번호'].head())
print("\n<raw> \n", raw_data['공개번호/공표/재공표 '].head(), "\n", raw_data['등록번호'].head())

<valid> 
 385    08796699
391    08788148
392    08786697
395    08781731
403    08723660
Name: 공개(등록)번호, dtype: object

<raw> 
 0    2014-0109386
1    2014-0111369
2    2010-0253540
3    2014-0083270
4    2014-0084089
Name: 공개번호/공표/재공표 , dtype: object 
 0        NaN
1        NaN
2    8704653
3        NaN
4        NaN
Name: 등록번호, dtype: object


In [189]:
df.loc[:, '공개번호/공표/재공표 '] = (df.loc[:, '공개번호/공표/재공표 '].str.split('-')).str.join('')
df.loc[:, '공개번호/공표/재공표 '].fillna(value = 0, inplace=True)

In [190]:
# 8자리 맞추기 ex)8704653 -> 08704653
# 00000000 -> 0

df.loc[:, '등록번호'] = df.loc[:, '등록번호'].apply(lambda x:'{0:0>8}'.format(x) if(np.all(pd.notnull(x))) else x).fillna(0)

In [191]:
df['등록번호'].head(10)

0           0
1           0
2    08704653
3           0
4           0
5           0
6           0
7    08674406
8           0
9    08669588
Name: 등록번호, dtype: object

### valid patent 확인<br>
US 유효특허 : 86개

In [192]:
#raw_data 중에서 val_data
y_idx_list = []
for idx, raw in enumerate(df["공개번호/공표/재공표 "]):
    for val in val_data["공개(등록)번호"]:
        if raw == val and idx not in y_idx_list:
            y_idx_list.append(idx)

for idx, raw in enumerate(df["등록번호"]):
    for val in val_data["공개(등록)번호"]:
        if raw == val and idx not in y_idx_list:
            y_idx_list.append(idx)

for idx, raw in enumerate(df["발명의 명칭"]):
    for val in val_data["발명의 명칭"]:
        if raw.lower() == val.lower() and idx not in y_idx_list:
            y_idx_list.append(idx)
y_data_list = [int(1) for i in range(len(y_idx_list))]
print(len(y_idx_list), len(y_data_list))

127 127


In [193]:
y_values = pd.Series(data=y_data_list, index=y_idx_list)
df.loc[:, "유효특허 여부"] = y_values
df["유효특허 여부"].fillna(0, inplace=True)

In [194]:
df['유효특허 여부'].value_counts()

0.0    1109
1.0     127
Name: 유효특허 여부, dtype: int64

<hr>

### IPC Digitization

In [195]:
df.loc[:, 'Original IPC Main'] = df.loc[:, 'Original IPC Main'].astype('str').str.slice(0,4)
df['Original IPC Main'].head()

0    B23P
1    G01S
2    B60Q
3    B28D
4    G09F
Name: Original IPC Main, dtype: object

In [196]:
# |로 묶인 IPC 처리
ipc_idx_list = []
ipc_all = []
for idx, i in enumerate(df['Original IPC All']):
    i = i.split('|')
    if len(i) != 1: # 여러 개 있으면
        ipc_list = []
        for ipc in i:
            ipc_list.append((ipc.strip())[:4])
        ipc_all.append(('|').join(ipc_list))
        ipc_idx_list.append(idx)
    else:
        ipc_all.append((i[0].strip())[0:4])
        ipc_idx_list.append(idx)

In [197]:
ipc_values = pd.Series(data = ipc_all, index=ipc_idx_list)
df.loc[:, 'Original IPC All'] = ipc_values
df['Original IPC All'].head()

0                        B23P
1                        G01S
2                   B60Q|G08G
3         B28D|B23K|B23Q|B26F
4    G09F|G01N|B07C|B02C|G09F
Name: Original IPC All, dtype: object

### fillna

In [198]:
df.loc[:, '출원인'] = df.loc[:, '출원인'].fillna(value='UKN')
df[['출원인']][60:70]

,출원인
60,"Samsung Electronics Co., Ltd."
61,UKN
62,"WMS Gaming, Inc."
63,Acushnet Company
64,"Immunolight, LLC"
65,"The Seaberg Company, Inc."
66,"Mitsubishi Electric Research Laboratories, Inc."
67,UKN
68,Xerox Corporation
69,"Magna Mirrors of America, Inc."


### including dash/slash data preprocessing

In [199]:
df.loc[:, '출원일'] = (df.loc[:, '출원일'].str.split('-')).str.join('')
df.loc[:, '출원일'] = df.loc[:, '출원일'].fillna(0)

df.loc[:, '공개일'] = (df.loc[:, '공개일'].str.split('-')).str.join('')
df.loc[:, '공개일'] = df.loc[:, '공개일'].fillna(0)

df.loc[:, '등록일'] = (df.loc[:, '등록일'].str.split('-')).str.join('')
df.loc[:, '등록일'] = df.loc[:, '등록일'].fillna(0)

df.loc[:, '국제 공개번호'] = (df.loc[:, '국제 공개번호'].str.split('-')).str.join('').str.slice(2,-1)
df.loc[:, '국제 공개번호'] = (df.loc[:, '국제 공개번호'].str.split('/')).str.join('')
df.loc[:, '국제 공개번호'] = df.loc[:, '국제 공개번호'].fillna(value=int(0))

### Label Encoding

In [200]:
application_id = list(df['출원번호'])
le = LabelEncoder()
le.fit(application_id)

application_id_le = le.transform(application_id)
df['출원번호'] = application_id_le

In [201]:
patent_kind = list(df['문헌종류 코드'])
le = LabelEncoder()
le.fit(patent_kind)

patent_kind_le = le.transform(patent_kind)
df['문헌종류 코드'] = patent_kind_le

In [202]:
priority_id = list(df['우선권 번호'])
le = LabelEncoder()
le.fit(priority_id)

priority_id_le = le.transform(priority_id)
df['우선권 번호'] = priority_id_le

In [204]:
df = df.astype('str')

In [205]:
df.to_csv('../data/night/data_night_1.csv', encoding='cp949')